In [1]:
import os
print("Current working directory:", os.getcwd())

Current working directory: /Users/ching-lung/deposit-prediction


## Data Preprocessing

This file is used for data pre-processing. The main goal is to merge the macroeconomic predictors with the targeted responses, including interest-bearing deposits, non-interesting-bearing deposits, and time deposits.

In [4]:
# imported packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
# Import the CSV file
dp = pd.read_csv("/Users/ching-lung/deposit-prediction/raw_data/predictors.csv")
dr = pd.read_csv("/Users/ching-lung/deposit-prediction/raw_data/responses.csv")

# Display the first few rows
dp.head(3)

,Scenario Name,Date,Real GDP growth,Nominal GDP growth,Real disposable income growth,Nominal disposable income growth,Unemployment rate,CPI inflation rate,3-month Treasury rate,5-year Treasury yield,10-year Treasury yield,BBB corporate yield,Mortgage rate,Prime rate,Dow Jones Total Stock Market Index (Level),House Price Index (Level),Commercial Real Estate Price Index (Level),Market Volatility Index (Level)
0,Actual,1976 Q1,9.3,14.0,5.0,9.6,7.7,4.7,4.9,7.4,7.6,NaN,8.9,6.8,NaN,22.9,50.9,NaN
1,Actual,1976 Q2,3.0,7.2,2.3,5.8,7.6,3.6,5.2,7.4,7.6,NaN,8.8,6.9,NaN,23.6,51.8,NaN
2,Actual,1976 Q3,2.2,7.6,3.2,9.6,7.7,6.5,5.2,7.3,7.6,NaN,9.0,7.1,NaN,24.2,52.6,NaN


In [6]:
# Initial economic values (in absolute numbers from 1976 Q1 data)
initial_gdp = {
    'Nominal GDP': 1688.6,  # $1,688.6 billions
    'Real GDP': 1676.0,     # $1,676.0 billions
    'Nominal disposable income': 1366.3,  # $1,366.3 billions
    'Real disposable income': 1358.0      # $1,358.0 billions
}

# Add initial GDP (and income) for each category with float64 type
gdp_columns = []
for category, initial_value in initial_gdp.items():
    gdp_column = f'{category} value'
    gdp_columns.append(gdp_column)
    dp[gdp_column] = pd.Series(dtype='float64')  # Ensure the column is float64
    dp.loc[0, gdp_column] = round(initial_value, 2)  # Set initial value for the first year

# Compute values iteratively for all categories
for category in initial_gdp.keys():
    gdp_column = f'{category} value'
    growth_column = f'{category} growth'
    for i in range(1, len(dp)):
        dp.loc[i, gdp_column] = round(dp.loc[i - 1, gdp_column] * (1 + dp.loc[i, growth_column] / 100), 2)

# Display the final DataFrame
dp.head(3)

,Scenario Name,Date,Real GDP growth,Nominal GDP growth,Real disposable income growth,Nominal disposable income growth,Unemployment rate,CPI inflation rate,3-month Treasury rate,5-year Treasury yield,...,Mortgage rate,Prime rate,Dow Jones Total Stock Market Index (Level),House Price Index (Level),Commercial Real Estate Price Index (Level),Market Volatility Index (Level),Nominal GDP value,Real GDP value,Nominal disposable income value,Real disposable income value
0,Actual,1976 Q1,9.3,14.0,5.0,9.6,7.7,4.7,4.9,7.4,...,8.9,6.8,NaN,22.9,50.9,NaN,1688.60,1676.00,1366.30,1358.00
1,Actual,1976 Q2,3.0,7.2,2.3,5.8,7.6,3.6,5.2,7.4,...,8.8,6.9,NaN,23.6,51.8,NaN,1810.18,1726.28,1445.55,1389.23
2,Actual,1976 Q3,2.2,7.6,3.2,9.6,7.7,6.5,5.2,7.3,...,9.0,7.1,NaN,24.2,52.6,NaN,1947.75,1764.26,1584.32,1433.69


In [7]:
dr.head(3)

,Assets and Liabilities of FDIC-Insured Commercial Banks and Savings Institutions,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 153,Unnamed: 154,Unnamed: 155,Unnamed: 156,Unnamed: 157,Unnamed: 158,Unnamed: 159,Unnamed: 160,Unnamed: 161,Unnamed: 162
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,(Amounts in $ Millions),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
dp['Date'] = dp['Date'].str.replace(' ', '')

In [9]:
dp

,Scenario Name,Date,Real GDP growth,Nominal GDP growth,Real disposable income growth,Nominal disposable income growth,Unemployment rate,CPI inflation rate,3-month Treasury rate,5-year Treasury yield,...,Mortgage rate,Prime rate,Dow Jones Total Stock Market Index (Level),House Price Index (Level),Commercial Real Estate Price Index (Level),Market Volatility Index (Level),Nominal GDP value,Real GDP value,Nominal disposable income value,Real disposable income value
0,Actual,1976Q1,9.3,14.0,5.0,9.6,7.7,4.7,4.9,7.4,...,8.9,6.8,NaN,22.9,50.9,NaN,1688.60,1676.00,1366.30,1358.00
1,Actual,1976Q2,3.0,7.2,2.3,5.8,7.6,3.6,5.2,7.4,...,8.8,6.9,NaN,23.6,51.8,NaN,1810.18,1726.28,1445.55,1389.23
2,Actual,1976Q3,2.2,7.6,3.2,9.6,7.7,6.5,5.2,7.3,...,9.0,7.1,NaN,24.2,52.6,NaN,1947.75,1764.26,1584.32,1433.69
3,Actual,1976Q4,2.9,10.5,2.6,9.2,7.8,5.9,4.7,6.5,...,8.8,6.5,NaN,25.2,53.4,NaN,2152.26,1815.42,1730.08,1470.97
4,Actual,1977Q1,4.8,11.7,0.9,8.4,7.5,7.5,4.6,6.8,...,8.7,6.3,NaN,26.2,55.0,NaN,2404.07,1902.56,1875.41,1484.21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,Actual,2022Q4,2.6,6.5,2.2,6.4,3.6,4.2,4.0,4.1,...,6.7,6.8,38520.6,295.9,350.0,33.6,73931334.56,244506.44,66070745.34,201871.07
188,Actual,2023Q1,2.2,6.3,10.8,15.5,3.5,3.8,4.6,3.8,...,6.4,7.7,41136.6,299.4,347.0,26.5,78589008.64,249885.58,76311710.87,223673.15
189,Actual,2023Q2,2.1,3.8,3.3,5.8,3.6,2.7,5.1,3.7,...,6.5,8.2,44411.5,303.0,354.0,20.1,81575390.97,255133.18,80737790.10,231054.36
190,Actual,2023Q3,4.9,8.3,0.3,2.9,3.7,3.6,5.3,4.3,...,7.0,8.4,42788.7,309.3,348.9,18.9,88346148.42,267634.71,83079186.01,231747.52


In [10]:
# Drop the first column
dp = dp.drop(dp.columns[0], axis=1)

In [11]:
dp

,Date,Real GDP growth,Nominal GDP growth,Real disposable income growth,Nominal disposable income growth,Unemployment rate,CPI inflation rate,3-month Treasury rate,5-year Treasury yield,10-year Treasury yield,...,Mortgage rate,Prime rate,Dow Jones Total Stock Market Index (Level),House Price Index (Level),Commercial Real Estate Price Index (Level),Market Volatility Index (Level),Nominal GDP value,Real GDP value,Nominal disposable income value,Real disposable income value
0,1976Q1,9.3,14.0,5.0,9.6,7.7,4.7,4.9,7.4,7.6,...,8.9,6.8,NaN,22.9,50.9,NaN,1688.60,1676.00,1366.30,1358.00
1,1976Q2,3.0,7.2,2.3,5.8,7.6,3.6,5.2,7.4,7.6,...,8.8,6.9,NaN,23.6,51.8,NaN,1810.18,1726.28,1445.55,1389.23
2,1976Q3,2.2,7.6,3.2,9.6,7.7,6.5,5.2,7.3,7.6,...,9.0,7.1,NaN,24.2,52.6,NaN,1947.75,1764.26,1584.32,1433.69
3,1976Q4,2.9,10.5,2.6,9.2,7.8,5.9,4.7,6.5,7.1,...,8.8,6.5,NaN,25.2,53.4,NaN,2152.26,1815.42,1730.08,1470.97
4,1977Q1,4.8,11.7,0.9,8.4,7.5,7.5,4.6,6.8,7.2,...,8.7,6.3,NaN,26.2,55.0,NaN,2404.07,1902.56,1875.41,1484.21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,2022Q4,2.6,6.5,2.2,6.4,3.6,4.2,4.0,4.1,3.9,...,6.7,6.8,38520.6,295.9,350.0,33.6,73931334.56,244506.44,66070745.34,201871.07
188,2023Q1,2.2,6.3,10.8,15.5,3.5,3.8,4.6,3.8,3.7,...,6.4,7.7,41136.6,299.4,347.0,26.5,78589008.64,249885.58,76311710.87,223673.15
189,2023Q2,2.1,3.8,3.3,5.8,3.6,2.7,5.1,3.7,3.7,...,6.5,8.2,44411.5,303.0,354.0,20.1,81575390.97,255133.18,80737790.10,231054.36
190,2023Q3,4.9,8.3,0.3,2.9,3.7,3.6,5.3,4.3,4.2,...,7.0,8.4,42788.7,309.3,348.9,18.9,88346148.42,267634.71,83079186.01,231747.52


In [12]:
subset = dr[dr['Assets and Liabilities of FDIC-Insured Commercial Banks and Savings Institutions'].isin(['Date', 'Credit cards', 'Loans to individuals', 'Other loans to individuals', 'Interest-bearing deposits', 'Noninterest-bearing deposits', 'Time deposits', 'Deposits'])]
subset

,Assets and Liabilities of FDIC-Insured Commercial Banks and Savings Institutions,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 153,Unnamed: 154,Unnamed: 155,Unnamed: 156,Unnamed: 157,Unnamed: 158,Unnamed: 159,Unnamed: 160,Unnamed: 161,Unnamed: 162
3,Date,1984Q1,1984Q2,1984Q3,1984Q4,1985Q1,1985Q2,1985Q3,1985Q4,1986Q1,...,2022Q1,2022Q2,2022Q3,2022Q4,2023Q1,2023Q2,2023Q3,2023Q4,2024Q1,2024Q2
16,Loans to individuals,"261,504","277,997","293,062","311,074","319,656","336,395","352,401","366,687","367,785",...,"1,881,272","1,961,748","2,001,120","2,070,581","2,037,736","2,072,175","2,096,296","2,135,614","2,085,639","2,111,443"
17,Credit cards,"46,521","51,159","56,645","64,110","67,028","71,279","75,943","83,691","85,172",...,"851,150","903,452","935,418","1,009,402","982,820","1,027,826","1,053,772","1,116,816","1,081,129","1,104,854"
18,Other loans to individuals,"214,984","226,839","236,417","246,963","252,628","265,116","276,458","282,996","282,613",...,"1,030,122","1,058,297","1,065,702","1,061,179","1,054,916","1,044,349","1,042,525","1,018,798","1,004,510","1,006,590"
50,Deposits,"2,697,459","2,773,230","2,781,986","2,907,668","2,916,647","2,984,331","3,051,300","3,140,827","3,163,361",...,"19,932,264","19,563,035","19,357,272","19,214,756","18,742,758","18,644,207","18,553,633","18,813,707","18,997,655","18,807,647"
53,Interest-bearing deposits,"1,860,467","1,921,613","1,964,713","2,043,037","2,086,014","2,124,432","2,162,307","2,216,165","2,394,640",...,"12,863,167","12,672,590","12,697,528","12,897,769","12,790,565","12,963,265","13,084,043","13,363,001","13,587,874","13,465,107"
54,Noninterest-bearing deposits,"363,956","379,300","364,125","421,536","371,462","400,009","424,541","460,092","430,845",...,"5,517,936","5,404,666","5,194,087","4,827,798","4,513,671","4,234,942","4,074,358","3,982,623","3,948,409","3,873,439"
56,Time deposits,"1,242,574","1,306,506","1,370,983","1,407,295","1,413,727","1,427,994","1,443,518","1,470,308","1,495,136",...,"1,246,782","1,322,152","1,486,596","1,708,631","2,133,081","2,439,754","2,685,279","2,869,517","2,938,551","2,980,053"


In [13]:
# Flip the DataFrame
flipped_df = subset.set_index('Assets and Liabilities of FDIC-Insured Commercial Banks and Savings Institutions').T.reset_index(drop=True)

In [14]:
# Drop the column name of the row index
flipped_df.columns.name = None
flipped_df

,Date,Loans to individuals,Credit cards,Other loans to individuals,Deposits,Interest-bearing deposits,Noninterest-bearing deposits,Time deposits
0,1984Q1,"261,504","46,521","214,984","2,697,459","1,860,467","363,956","1,242,574"
1,1984Q2,"277,997","51,159","226,839","2,773,230","1,921,613","379,300","1,306,506"
2,1984Q3,"293,062","56,645","236,417","2,781,986","1,964,713","364,125","1,370,983"
3,1984Q4,"311,074","64,110","246,963","2,907,668","2,043,037","421,536","1,407,295"
4,1985Q1,"319,656","67,028","252,628","2,916,647","2,086,014","371,462","1,413,727"
...,...,...,...,...,...,...,...,...
157,2023Q2,"2,072,175","1,027,826","1,044,349","18,644,207","12,963,265","4,234,942","2,439,754"
158,2023Q3,"2,096,296","1,053,772","1,042,525","18,553,633","13,084,043","4,074,358","2,685,279"
159,2023Q4,"2,135,614","1,116,816","1,018,798","18,813,707","13,363,001","3,982,623","2,869,517"
160,2024Q1,"2,085,639","1,081,129","1,004,510","18,997,655","13,587,874","3,948,409","2,938,551"


In [15]:
# Merge DataFrames with respect to the first DataFrame
merge_data = pd.merge(dp, flipped_df, on='Date', how='right')  # Merge but keep all rows from df1

In [16]:
merge_data

,Date,Real GDP growth,Nominal GDP growth,Real disposable income growth,Nominal disposable income growth,Unemployment rate,CPI inflation rate,3-month Treasury rate,5-year Treasury yield,10-year Treasury yield,...,Real GDP value,Nominal disposable income value,Real disposable income value,Loans to individuals,Credit cards,Other loans to individuals,Deposits,Interest-bearing deposits,Noninterest-bearing deposits,Time deposits
0,1984Q1,8.1,12.5,7.9,12.7,7.9,5.8,9.2,11.7,11.9,...,4370.98,30668.53,3494.10,"261,504","46,521","214,984","2,697,459","1,860,467","363,956","1,242,574"
1,1984Q2,7.1,10.8,6.6,10.8,7.4,3.8,9.8,13.0,13.2,...,4681.32,33980.73,3724.71,"277,997","51,159","226,839","2,773,230","1,921,613","379,300","1,306,506"
2,1984Q3,3.9,7.7,5.3,8.6,7.4,3.5,10.3,12.8,12.9,...,4863.89,36903.07,3922.12,"293,062","56,645","236,417","2,781,986","1,964,713","364,125","1,370,983"
3,1984Q4,3.3,6.4,3.4,5.9,7.3,3.5,8.8,11.5,11.8,...,5024.40,39080.35,4055.47,"311,074","64,110","246,963","2,907,668","2,043,037","421,536","1,407,295"
4,1985Q1,3.9,8.1,-1.0,3.8,7.2,3.7,8.2,11.3,11.6,...,5220.35,40565.40,4014.92,"319,656","67,028","252,628","2,916,647","2,086,014","371,462","1,413,727"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,2023Q2,2.1,3.8,3.3,5.8,3.6,2.7,5.1,3.7,3.7,...,255133.18,80737790.10,231054.36,"2,072,175","1,027,826","1,044,349","18,644,207","12,963,265","4,234,942","2,439,754"
158,2023Q3,4.9,8.3,0.3,2.9,3.7,3.6,5.3,4.3,4.2,...,267634.71,83079186.01,231747.52,"2,096,296","1,053,772","1,042,525","18,553,633","13,084,043","4,074,358","2,685,279"
159,2023Q4,1.5,3.6,2.2,4.4,3.7,2.8,5.3,4.5,4.5,...,271649.23,86734670.19,236845.97,"2,135,614","1,116,816","1,018,798","18,813,707","13,363,001","3,982,623","2,869,517"
160,2024Q1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,"2,085,639","1,081,129","1,004,510","18,997,655","13,587,874","3,948,409","2,938,551"


In [17]:
data_clean = merge_data.iloc[:-2].dropna(axis=1)
data_clean

,Date,Real GDP growth,Nominal GDP growth,Real disposable income growth,Nominal disposable income growth,Unemployment rate,CPI inflation rate,3-month Treasury rate,5-year Treasury yield,10-year Treasury yield,...,Real GDP value,Nominal disposable income value,Real disposable income value,Loans to individuals,Credit cards,Other loans to individuals,Deposits,Interest-bearing deposits,Noninterest-bearing deposits,Time deposits
0,1984Q1,8.1,12.5,7.9,12.7,7.9,5.8,9.2,11.7,11.9,...,4370.98,30668.53,3494.10,"261,504","46,521","214,984","2,697,459","1,860,467","363,956","1,242,574"
1,1984Q2,7.1,10.8,6.6,10.8,7.4,3.8,9.8,13.0,13.2,...,4681.32,33980.73,3724.71,"277,997","51,159","226,839","2,773,230","1,921,613","379,300","1,306,506"
2,1984Q3,3.9,7.7,5.3,8.6,7.4,3.5,10.3,12.8,12.9,...,4863.89,36903.07,3922.12,"293,062","56,645","236,417","2,781,986","1,964,713","364,125","1,370,983"
3,1984Q4,3.3,6.4,3.4,5.9,7.3,3.5,8.8,11.5,11.8,...,5024.40,39080.35,4055.47,"311,074","64,110","246,963","2,907,668","2,043,037","421,536","1,407,295"
4,1985Q1,3.9,8.1,-1.0,3.8,7.2,3.7,8.2,11.3,11.6,...,5220.35,40565.40,4014.92,"319,656","67,028","252,628","2,916,647","2,086,014","371,462","1,413,727"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,2022Q4,2.6,6.5,2.2,6.4,3.6,4.2,4.0,4.1,3.9,...,244506.44,66070745.34,201871.07,"2,070,581","1,009,402","1,061,179","19,214,756","12,897,769","4,827,798","1,708,631"
156,2023Q1,2.2,6.3,10.8,15.5,3.5,3.8,4.6,3.8,3.7,...,249885.58,76311710.87,223673.15,"2,037,736","982,820","1,054,916","18,742,758","12,790,565","4,513,671","2,133,081"
157,2023Q2,2.1,3.8,3.3,5.8,3.6,2.7,5.1,3.7,3.7,...,255133.18,80737790.10,231054.36,"2,072,175","1,027,826","1,044,349","18,644,207","12,963,265","4,234,942","2,439,754"
158,2023Q3,4.9,8.3,0.3,2.9,3.7,3.6,5.3,4.3,4.2,...,267634.71,83079186.01,231747.52,"2,096,296","1,053,772","1,042,525","18,553,633","13,084,043","4,074,358","2,685,279"


In [18]:
# Remove commas from all columns except the first one
data_clean.iloc[:, 1:] = data_clean.iloc[:, 1:].replace(',', '', regex=True)

# Convert all columns except the first to float64
data_clean.iloc[:, 1:] = data_clean.iloc[:, 1:].apply(pd.to_numeric, errors='coerce')

In [19]:
data_clean

,Date,Real GDP growth,Nominal GDP growth,Real disposable income growth,Nominal disposable income growth,Unemployment rate,CPI inflation rate,3-month Treasury rate,5-year Treasury yield,10-year Treasury yield,...,Real GDP value,Nominal disposable income value,Real disposable income value,Loans to individuals,Credit cards,Other loans to individuals,Deposits,Interest-bearing deposits,Noninterest-bearing deposits,Time deposits
0,1984Q1,8.1,12.5,7.9,12.7,7.9,5.8,9.2,11.7,11.9,...,4370.98,30668.53,3494.10,261504,46521,214984,2697459,1860467,363956,1242574
1,1984Q2,7.1,10.8,6.6,10.8,7.4,3.8,9.8,13.0,13.2,...,4681.32,33980.73,3724.71,277997,51159,226839,2773230,1921613,379300,1306506
2,1984Q3,3.9,7.7,5.3,8.6,7.4,3.5,10.3,12.8,12.9,...,4863.89,36903.07,3922.12,293062,56645,236417,2781986,1964713,364125,1370983
3,1984Q4,3.3,6.4,3.4,5.9,7.3,3.5,8.8,11.5,11.8,...,5024.40,39080.35,4055.47,311074,64110,246963,2907668,2043037,421536,1407295
4,1985Q1,3.9,8.1,-1.0,3.8,7.2,3.7,8.2,11.3,11.6,...,5220.35,40565.40,4014.92,319656,67028,252628,2916647,2086014,371462,1413727
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,2022Q4,2.6,6.5,2.2,6.4,3.6,4.2,4.0,4.1,3.9,...,244506.44,66070745.34,201871.07,2070581,1009402,1061179,19214756,12897769,4827798,1708631
156,2023Q1,2.2,6.3,10.8,15.5,3.5,3.8,4.6,3.8,3.7,...,249885.58,76311710.87,223673.15,2037736,982820,1054916,18742758,12790565,4513671,2133081
157,2023Q2,2.1,3.8,3.3,5.8,3.6,2.7,5.1,3.7,3.7,...,255133.18,80737790.10,231054.36,2072175,1027826,1044349,18644207,12963265,4234942,2439754
158,2023Q3,4.9,8.3,0.3,2.9,3.7,3.6,5.3,4.3,4.2,...,267634.71,83079186.01,231747.52,2096296,1053772,1042525,18553633,13084043,4074358,2685279


In [22]:
data_clean.to_csv("/Users/ching-lung/deposit-prediction/cleaned_data/data_clean.csv", index=False)